In [ ]:
#based off thishttps://towardsdatascience.com/sentiment-analysis-using-lstm-step-by-step-50d074f09948
#https://machinetalk.org/2019/03/18/introduction-to-tensorflow-datasets/
#my chemistry senior project

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import Counter
import os

total = 512

def file_reader():
    
    batch_size = 16
    seq_size = 32
    
    with open('/Users/weichiang/Desktop/warpeace_input.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    text = text.split()
    counter = Counter(text)
    dict_counter = dict(counter)
    v = sorted(list(dict_counter.items()), key=lambda item: item[1], reverse = True)
    v_sorted = [x[0] for x in v]
    enum_int_vocab = {k: w for k, w in enumerate(v_sorted)}
    enum_vocab_int = {w: k for k, w in enum_int_vocab.items()}
    print (len(enum_int_vocab))

    mega = [enum_vocab_int[article] for article in text]
    total_batches = int(len(mega) / (seq_size * batch_size))
    
    #wanted to use total batches so divisible by 8/16/32...
    amnt_mega = total_batches*batch_size * seq_size
    text = mega[:amnt_mega]
    nptext = np.zeros_like(text)
    index = 1
    
    for i in reversed(range(len(nptext)-1)):
        nptext[i] = text[index]
        index +=1
    nptext[-1] = text[0]

    text = np.reshape(text, (batch_size, -1))
    nptext = np.reshape(nptext, (batch_size, -1))
    
    return enum_int_vocab, enum_vocab_int, text, nptext, len(enum_int_vocab)

In [2]:
def batch_gen(text, nptext, batch_size, seq_size):
    for i in range(0, np.prod(text.shape) // (total) * seq_size, seq_size):
        yield text[:, i:i+seq_size], nptext[:, i:i+seq_size]

In [3]:
class RNN(nn.Module):
#     length = len(enum_int_vocab)
#embedding_dim = 64
#layers = 2
#size = lstm_size
    def __init__(self, length, embedding_dim, layers, size):
        super(RNN, self).__init__()
        self.size = size
        self.embedding = nn.Embedding(length, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,layers,batch_first=True,dropout = 0.2)
        self.dense = nn.Linear(size, length)

    def forward(self, x, prev_state):
        e = self.embedding(x)
        output, state = self.lstm(e, prev_state)
        result = self.dense(output)

        return result, state

In [4]:
def predict(net, length, enum_vocab_int, enum_int_vocab):
    net.eval()
    start = ['I', 'is']
    criterion, optimizer = nn.CrossEntropyLoss(), torch.optim.Adam(net.parameters(), 0.001)
    
    hidden_state, state = torch.zeros(1, 1, net.size), torch.zeros(1, 1, net.size)

    for words in start:
        output, (hidden_state, state) = net(torch.tensor([[enum_vocab_int[words]]]), (hidden_state, state))

    blank, top_pick = torch.topk(output[0], k=5)
    choice = np.random.choice(top_pick.tolist()[0])

    start.append(enum_int_vocab[choice])

    for x in range(100):
        output, (hidden_state, state) = net(torch.tensor([[choice]]), (hidden_state, state))

        x, top_pick = torch.topk(output[0], k=5)
        choices = top_pick.tolist()
        choice = np.random.choice(choices[0])
        start.append(enum_int_vocab[choice])

    print(' '.join(words).encode('utf-8'))

In [7]:
def main():
    enum_int_vocab, enum_vocab_int, text, nptext, length = file_reader()
    net = RNN(length, 32, 64, 64)
    criterion, optimizer = nn.CrossEntropyLoss(), torch.optim.Adam(net.parameters(), 0.001)
    index = 0

    for i in range(200):
        batches = batch_gen(text, nptext, 16, 32)
        hidden_state, state = torch.zeros(1, 16, net.size), torch.zeros(1, 16, net.size)
        
        for x, y in batches:
            index += 1
            net.train()

            optimizer.zero_grad()

            x = torch.tensor(x)
            y = torch.tensor(y)

            result, (hidden_state, state) = net(x, (hidden_state, state))
            loss = criterion(result.transpose(1, 2), y)
            loss.backward()


            hidden_state = hidden_state.detach()
            state = state.detach()

            _ = torch.nn.utils.clip_grad_norm_(net.parameters(), 5)
            optimizer.step()
            
            if index % 10 == 0:
                print('Epoch: {}/{}'.format(i, 200),
                      'Iteration: {}'.format(index),
                      'Loss: {}'.format(loss.item()))
                
            if index % 10 == 0:
                predict(net, length, enum_vocab_int, enum_int_vocab)

In [ ]:

if __name__ == '__main__':
    main()

41566
Epoch: 0/200 Iteration: 10 Loss: 10.603655815124512
b'i s'
Epoch: 0/200 Iteration: 20 Loss: 10.49414348602295
b'i s'
Epoch: 0/200 Iteration: 30 Loss: 9.81558609008789
b'i s'
